In [1]:
import requests
import json
import os
from dotenv import load_dotenv
from urllib.parse import quote

# 환경 설정
load_dotenv()
service_key = quote(os.getenv("SMARTFARM_API_KEY").strip(), safe='')

# 실제 작동하는 제어 정보 API
def get_cropping_season_control_data(service_key, cropping_serl_no, page_num=1):
    """작기별 제어 정보 조회"""
    url = f"https://www.smartfarmkorea.net/Agree_WS/webservices/CropseasonRestService/getCroppingSeasonManlDataList/{service_key}/{cropping_serl_no}/{page_num}"
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        data = response.json()
        return data if data else None
    except Exception as e:
        print(f"❌ 제어 정보 실패: {e}")
        return None

# 작기 정보 조회 함수
def get_cropping_season_data_list(service_key, user_id):
    """작기 정보 조회"""
    url = f"http://www.smartfarmkorea.net/Agree_WS/webservices/ProvideRestService/getCroppingSeasonDataList/{service_key}/{user_id}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ 작기 정보 실패: {e}")
        return None

print("✅ API 함수 준비 완료")


✅ API 함수 준비 완료


In [13]:
# PF_0020745 농가 제어 정보 수집 (중복 방지 개선 버전)
target_user_id = "PF_0020745"

print(f"🚀 {target_user_id} 농가 제어 정보 수집 시작...")
print("=" * 70)

# ✅ 1. 변수 완전 초기화 (중복 방지)
all_control_data = []  # 리스트 완전 초기화
total_records = 0      # 카운터 초기화
cropping_data = None   # 작기 데이터 초기화

print("🔄 변수 초기화 완료")

# ✅ 2. 기존 파일 확인
expected_filename = f"./data/{target_user_id}_complete_control_data.json"
if os.path.exists(expected_filename):
    print(f"⚠️  기존 파일 발견: {expected_filename}")
    file_size = os.path.getsize(expected_filename) / 1024 / 1024  # MB 단위
    print(f"   파일 크기: {file_size:.2f} MB")
    print(f"   새로 수집하면 기존 파일이 덮어쓰여집니다.")

# ✅ 3. 작기 정보 조회
print("\n📋 작기 정보 조회 중...")
try:
    cropping_data = get_cropping_season_data_list(service_key, target_user_id)
    
    if not cropping_data or len(cropping_data) == 0:
        print("❌ 작기 정보를 조회할 수 없습니다.")
        cropping_data = None
    else:
        print(f"✅ 작기 정보 조회 성공: {len(cropping_data)}개 작기 발견")
        
        # 작기 정보 상세 출력
        print("\\n📊 발견된 작기들:")
        for i, crop in enumerate(cropping_data):
            serl_no = crop.get('croppingSerlNo')
            start_date = crop.get('croppingDate')
            end_date = crop.get('croppingEndDate')
            item_code = crop.get('itemCode')
            print(f"  [{i+1}] 작기 {serl_no}: {start_date} ~ {end_date} (품목: {item_code})")
            
except Exception as e:
    print(f"❌ 작기 정보 조회 실패: {e}")
    cropping_data = None

# ✅ 4. 제어 정보 수집 (첫 번째 작기만 - 인덱스 수정)
if cropping_data and len(cropping_data) > 0:
    print("\\n🔧 첫 번째 작기 제어 정보 수집 중...")
    
    # ⚡ 수정: cropping_data[1] → cropping_data[0] (첫 번째 작기)
    first_crop = cropping_data[2]  # 🔴 인덱스 수정!
    serl_no = first_crop.get('croppingSerlNo')
    
    if serl_no:
        print(f"\\n📊 작기 {serl_no} 처리 중...")
        
        # ✅ 5. 작기별 데이터 수집 (로컬 변수 사용)
        crop_control_data = []  # 지역 변수로 초기화
        page = 1
        page_limit = 100  # 안전장치: 최대 20페이지
        
        while page <= page_limit:
            print(f"  📄 페이지 {page} 수집 중...", end=" ")
            
            try:
                # API 호출
                control_data = get_cropping_season_control_data(service_key, serl_no, page)
                
                if not control_data or not isinstance(control_data, list) or len(control_data) == 0:
                    print("❌ 데이터 없음 - 수집 완료")
                    break
                
                # 페이지 정보 확인
                first_record = control_data[0] if control_data else {}
                total_pages = first_record.get('totalPage', '1')
                total_rows = first_record.get('totalRows', len(control_data))
                
                print(f"✅ {len(control_data)}개 레코드 (전체: {total_rows}개, {total_pages}페이지 중 {page}페이지)")
                
                # ✅ 6. 데이터 누적 (안전하게)
                crop_control_data.extend(control_data)
                total_records += len(control_data)
                
                # 마지막 페이지 확인
                try:
                    if int(page) >= int(total_pages):
                        print(f"    📝 마지막 페이지 도달 - 작기 {serl_no} 수집 완료")
                        break
                except ValueError:
                    if len(control_data) < 1000:
                        print(f"    📝 데이터 개수 기준으로 마지막 페이지 추정")
                        break
                
                page += 1
                
            except Exception as e:
                print(f"❌ 페이지 {page} 수집 실패: {e}")
                break
        
        # ✅ 7. 수집된 데이터 저장
        if crop_control_data:
            all_control_data.append({
                'userInfo': {
                    'userId': target_user_id,
                    'croppingSerlNo': serl_no,
                    'croppingDate': first_crop.get('croppingDate'),
                    'croppingEndDate': first_crop.get('croppingEndDate'),
                    'itemCode': first_crop.get('itemCode')
                },
                'controlData': crop_control_data,
                'totalRecords': len(crop_control_data)
            })
            print(f"  ✅ 작기 {serl_no}: {len(crop_control_data):,}개 제어 레코드 수집 완료")
        else:
            print(f"  ❌ 작기 {serl_no}: 수집된 데이터 없음")
    else:
        print("❌ 첫 번째 작기의 일련번호를 찾을 수 없습니다.")
else:
    print("❌ 수집할 작기 정보가 없습니다.")

# ✅ 8. 최종 결과 출력
print(f"\\n📈 수집 결과 요약:")
print(f"  🌱 처리된 작기 수: {len(all_control_data)}개")
print(f"  📊 총 제어 레코드: {total_records:,}개")

if all_control_data:
    print(f"  ✅ 데이터 수집 성공!")
    
    # 샘플 제어 코드 확인
    if all_control_data[0]['controlData']:
        sample_data = all_control_data[0]['controlData'][:5]
        fatr_codes = set()
        for record in sample_data:
            if isinstance(record, dict) and 'fatrCode' in record:
                fatr_codes.add(record['fatrCode'])
        print(f"  🔍 발견된 제어 코드 샘플: {sorted(list(fatr_codes))}")
else:
    print(f"  ❌ 수집된 데이터 없음")

print(f"\\n🎯 다음 셀에서 파일 저장을 진행하세요!")


🚀 PF_0020745 농가 제어 정보 수집 시작...
🔄 변수 초기화 완료
⚠️  기존 파일 발견: ./data/PF_0020745_complete_control_data.json
   파일 크기: 12.12 MB
   새로 수집하면 기존 파일이 덮어쓰여집니다.

📋 작기 정보 조회 중...
✅ 작기 정보 조회 성공: 5개 작기 발견
\n📊 발견된 작기들:
  [1] 작기 4434: 2020-10-24 ~ 2021-06-01 (품목: 080300)
  [2] 작기 4557: 2021-05-17 ~ 2021-09-17 (품목: 090100)
  [3] 작기 4862: 2021-09-02 ~ 2022-06-10 (품목: 080300)
  [4] 작기 5698: 2020-08-01 ~ 2022-12-31 (품목: 080300)
  [5] 작기 6022: 2022-09-19 ~ 2023-06-03 (품목: 080300)
\n🔧 첫 번째 작기 제어 정보 수집 중...
\n📊 작기 4862 처리 중...
  📄 페이지 1 수집 중... ✅ 1000개 레코드 (전체: 76905개, 77페이지 중 1페이지)
  📄 페이지 2 수집 중... ✅ 1000개 레코드 (전체: 76905개, 77페이지 중 2페이지)
  📄 페이지 3 수집 중... ✅ 1000개 레코드 (전체: 76905개, 77페이지 중 3페이지)
  📄 페이지 4 수집 중... ✅ 1000개 레코드 (전체: 76905개, 77페이지 중 4페이지)
  📄 페이지 5 수집 중... ✅ 1000개 레코드 (전체: 76905개, 77페이지 중 5페이지)
  📄 페이지 6 수집 중... ✅ 1000개 레코드 (전체: 76905개, 77페이지 중 6페이지)
  📄 페이지 7 수집 중... ✅ 1000개 레코드 (전체: 76905개, 77페이지 중 7페이지)
  📄 페이지 8 수집 중... ✅ 1000개 레코드 (전체: 76905개, 77페이지 중 8페이지)
  📄 페이지 9 수집 중... ✅ 1000개 레코드 (전체: 7

In [14]:
# 데이터 저장 (별도 셀로 분리)
if 'all_control_data' in globals() and all_control_data:
    print("💾 제어 데이터 파일 저장 중...")
    print("=" * 50)
    
    try:
        # 데이터 폴더 생성
        os.makedirs("./data", exist_ok=True)
        
        # 첫 번째 작기 정보 가져오기
        first_season = all_control_data[0]
        user_info = first_season['userInfo']
        serl_no = user_info['croppingSerlNo']
        
        # 파일명 생성 (작기 번호 포함)
        filename = f"./data/{target_user_id}_{serl_no}_control_data.json"
        summary_filename = f"./data/{target_user_id}_{serl_no}_control_summary.json"
        
        # 전체 데이터 저장
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(all_control_data, f, ensure_ascii=False, indent=2)
        
        file_size = os.path.getsize(filename) / 1024 / 1024  # MB 단위
        print(f"✅ 제어 데이터 저장 완료: {filename}")
        print(f"   파일 크기: {file_size:.2f} MB")
        
        # 요약 정보 생성 및 저장
        from datetime import datetime
        summary = {
            'userId': target_user_id,
            'collectionDateTime': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'totalSeasons': len(all_control_data),
            'totalRecords': sum(season['totalRecords'] for season in all_control_data),
            'seasons': []
        }
        
        for data in all_control_data:
            user_info = data['userInfo']
            summary['seasons'].append({
                'croppingSerlNo': user_info['croppingSerlNo'],
                'period': f"{user_info['croppingDate']} ~ {user_info['croppingEndDate']}",
                'itemCode': user_info['itemCode'],
                'recordCount': data['totalRecords']
            })
        
        with open(summary_filename, "w", encoding="utf-8") as f:
            json.dump(summary, f, ensure_ascii=False, indent=2)
        
        print(f"✅ 요약 정보 저장 완료: {summary_filename}")
        
        # 최종 성공 메시지
        print(f"\n🎉 {target_user_id} 농가 제어 데이터 수집 및 저장 완료!")
        print(f"📈 최종 결과:")
        print(f"  🌱 작기 수: {len(all_control_data)}개")
        print(f"  📊 총 제어 레코드: {summary['totalRecords']:,}개")
        print(f"  📁 저장 파일:")
        print(f"    - 전체 데이터: {filename}")
        print(f"    - 요약 정보: {summary_filename}")
        
        # 작기별 상세 정보
        print(f"\n📋 작기별 상세 정보:")
        for data in all_control_data:
            user_info = data['userInfo']
            print(f"  작기 {user_info['croppingSerlNo']}: {data['totalRecords']:,}개 레코드")
            print(f"    기간: {user_info['croppingDate']} ~ {user_info['croppingEndDate']}")
            print(f"    품목: {user_info['itemCode']}")
        
        # 제어 코드 분석
        if all_control_data[0]['controlData']:
            all_codes = set()
            for season_data in all_control_data:
                for record in season_data['controlData']:
                    if isinstance(record, dict) and 'fatrCode' in record:
                        all_codes.add(record['fatrCode'])
            
            print(f"\n🔍 수집된 제어 코드 전체: {len(all_codes)}개")
            print(f"  코드 목록: {sorted(list(all_codes))}")
        
        # 변수 정리 (메모리 절약)
        print(f"\n🧹 변수 정리 완료 - 메모리 절약")
        
    except Exception as e:
        print(f"❌ 파일 저장 실패: {e}")
        
else:
    print("❌ 저장할 데이터가 없습니다.")
    print("💡 먼저 이전 셀을 실행해서 데이터를 수집해주세요.")


💾 제어 데이터 파일 저장 중...
✅ 제어 데이터 저장 완료: ./data/PF_0020745_4862_control_data.json
   파일 크기: 33.45 MB
✅ 요약 정보 저장 완료: ./data/PF_0020745_4862_control_summary.json

🎉 PF_0020745 농가 제어 데이터 수집 및 저장 완료!
📈 최종 결과:
  🌱 작기 수: 1개
  📊 총 제어 레코드: 76,905개
  📁 저장 파일:
    - 전체 데이터: ./data/PF_0020745_4862_control_data.json
    - 요약 정보: ./data/PF_0020745_4862_control_summary.json

📋 작기별 상세 정보:
  작기 4862: 76,905개 레코드
    기간: 2021-09-02 ~ 2022-06-10
    품목: 080300

🔍 수집된 제어 코드 전체: 14개
  코드 목록: ['CC01_1_L_V', 'CC01_1_R_V', 'CC01_2_L_V', 'CC01_2_R_V', 'CC03_1_L_V', 'CC03_1_R_V', 'CC0403', 'CC0803', 'CC1103', 'CC1403', 'CC1705', 'CC1803', 'CC2403', 'CC3402']

🧹 변수 정리 완료 - 메모리 절약


In [15]:
# 변수 정리 및 메모리 해제 (선택적 실행)
print("🧹 메모리 정리 중...")

# 크기가 큰 변수들 정리
if 'all_control_data' in globals():
    data_size = len(str(all_control_data)) / 1024 / 1024  # 대략적인 메모리 크기
    print(f"📊 all_control_data 메모리 사용량: 약 {data_size:.2f} MB")
    
    # 사용자 선택에 따라 변수 삭제
    # del all_control_data  # 주석 해제하면 변수 삭제

if 'cropping_data' in globals():
    # del cropping_data  # 주석 해제하면 변수 삭제
    pass

# 현재 전역 변수 목록 확인
control_vars = [var for var in globals().keys() if 'control' in var.lower() or 'crop' in var.lower()]
print(f"🔍 제어/작기 관련 변수들: {control_vars}")

print("✅ 정리 완료! 필요시 위의 del 명령어 주석을 해제하여 메모리를 해제할 수 있습니다.")
print("💡 변수 삭제 후에는 다시 데이터 수집을 실행해야 합니다.")


🧹 메모리 정리 중...
📊 all_control_data 메모리 사용량: 약 23.62 MB
🔍 제어/작기 관련 변수들: ['get_cropping_season_control_data', 'get_cropping_season_data_list', 'all_control_data', 'cropping_data', 'crop', 'first_crop', 'crop_control_data', 'control_data', 'control_vars']
✅ 정리 완료! 필요시 위의 del 명령어 주석을 해제하여 메모리를 해제할 수 있습니다.
💡 변수 삭제 후에는 다시 데이터 수집을 실행해야 합니다.


In [ ]:
with open("./data/PF_0020745_4434_control_data.json", "r", encoding="utf-8") as f:
        control_data = json.load(f)